# Camera pose and orientation estimation from QR code images given real world coordinates.

### This application visualises the camera position and orientation in the real world. As inputs you need to give in an image whose camera location we need to estimate, the original image and its dimensions. 

![](/multi.jpg)

### How to run the program:
\>>> python pose.py [input_images_dir_path] \[original_image_path\] \[original_imgage_dimension\]

## Searching for the QR code in the given image
One assumption made on the basis of the input data is that the input image only has a single QR code lying on an otherwise plain background. 

In [ ]:
img_path = '1.jpg'
import numpy as np
import cv2
image = cv2.imread(img_path)

### Finding Contours
First convert the image to greyscale, apply a threshold to get binary image and then apply Canny edge detector. Finally find the contours in the resultant image.

In [ ]:
def get_hier_contours(image):
    """
    Returns the contours and their heirarachy for a given image.
    """
    # Constants
    thresh = 200
    thresh_max = 255
    canny_min = 100
    canny_max = 200

    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    th, thresh_img = cv2.threshold(gray_img, thresh, thresh_max, cv2.THRESH_BINARY)
    canny = cv2.Canny(thresh_img, canny_min, canny_max)
    _, contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy.squeeze()
    return hierarchy, contours

In [ ]:
hierarchy, contours = get_hier_contours(image)

![](canny.jpg)
<center>Canny edge detection.</center>

### Selecting contours that identify the QR code's position markers
Loop through all the contours and check each contour for 6 nesting levels to identify the QR code markers.

In [ ]:
class Contour:    
    """
    Represents a contour with its number, the contour data and its centroid.
    """
    def __init__(self, num, contour, centroid):
        self.num = num
        self.contour = contour
        self.centroid = centroid

In [ ]:
def get_centroid(contour):
    """
    Returns the centroid for a given contour.
    """
    m = cv2.moments(contour)
    if m['m00'] != 0:
        centroid_x =int(m['m10']/m['m00'])
        centroid_y =int(m['m01']/m['m00'])
    return np.array((centroid_x, centroid_y))

In [ ]:
def get_position_markers(hierarchy, contours):
    """
    Returns an array of Contour classes for the position markers of the QR code. 
    """
    #Constants
    nesting_max = 5
    conts = []
    for h in range(len(hierarchy)):
        i = h 
        nesting = 0
        while hierarchy[i][2] != -1:
            i = hierarchy[i][2]
            nesting += 1
        if nesting == nesting_max:
            conts.append(Contour(h, contours[h], get_centroid(contours[h])))
#     corner_marker_contours = np.asarray((conts[0].contour,conts[1].contour,conts[2].contour))
    return np.asarray(conts)

In [ ]:
conts = get_position_markers(hierarchy, contours)

### Get the relative location of each of the points obtained
Going by the conventional structure of a QR code, the bottom right does not have a position marker. So from the 3 points obtained, the 2 points that are equidistant from the third will be opposite corners and the third point will be the top left corner.

In [ ]:
def get_dist(point_a, point_b):
    """
    Returns the distance between two 2D coordinates.
    """
    a = point_a.astype(np.float32)
    b = point_b.astype(np.float32)
    return np.linalg.norm(a - b)

In [ ]:
def vert_opp_largest_side(C1, C2, C3):
    """
    Given 3 points(Contours in this case), returns the 3 contours rearranged
    in such a way that the cetroid of first returned contour is opposite to the largest side 
    in the triangle formed by the three centroids. 
    """
    P1 = C1.centroid
    P2 = C2.centroid
    P3 = C3.centroid
    
    P1P2 = get_dist(P1, P2)
    P2P3 = get_dist(P2, P3)
    P3P1 = get_dist(P3, P1)
    
    if P1P2 > P2P3 and P1P2 > P3P1:
        return [C3, C1, C2]
    elif P2P3 > P1P2 and P2P3 > P3P1:
        return [C1, C2, C3]
    elif P3P1 > P1P2 and P3P1 > P2P3:
        return [C2, C1, C3]

#### Points representing centroid of the position markers

In [ ]:
A, B, C = vert_opp_largest_side(conts[0], conts[1], conts[2])
print(A.centroid,B.centroid,C.centroid)

#### Get the centroid of the pattern for the 4th point
Since OpenCVs API to find extrinsic matrices (translational and rotational) requires a minimum of 4 points, we can select the midpoint of the QR code as the fourth point.Beings a prspective projection, the centroid will be the center of line joiing any two extreme vertices of the square pattern. We have 3 vertices such that, 1 is not a part of the diagonal, so the remaining two can be used to find centroid. (This is an approximation. Since it is a perspective projection, the actual midpoint of the QR code wont lie on the midpoint of the diagonal. But because the subject is not at a very large distance form the camera, this approximation will give considerably good results.)

In [ ]:
def get_midpoint(point_a, point_b):
    """
    Returns the midpoint of the line segment joining the given 2 points.
    """
    return np.asarray(((point_a[0]+point_b[0])/2, (point_a[1]+point_b[1])/2))

In [ ]:
mid = get_midpoint(B.centroid, C.centroid)
print(B.centroid,C.centroid,mid)

#### Get extreme vertices of position markers
The coordinates obtained from the contours A, B and C's represent their centroid. Since we need to get as accurate pose of the camera as possible, we will need to get the extreme corners of these contours of the position markers as that is the measurement we know (8.8cm)

In [ ]:
def get_max_dist(cnt, point):
    """
    Returns the pointfrom the contour that is farthest from the given point.
    """
    l = np.asarray(cnt[cnt[:,:,0].argmin()][0])
    r = np.asarray(cnt[cnt[:,:,0].argmax()][0])
    t = np.asarray(cnt[cnt[:,:,1].argmin()][0])
    b = np.asarray(cnt[cnt[:,:,1].argmax()][0])
    
    req = [l,r,t,b]
    
    max_dist = -1;
    max_pts = point
    for p in req:
        dist = get_dist(p, point)
        if max_dist < dist:
            max_dist = dist
            max_pts = p
    return max_pts.squeeze()

In [ ]:
x = get_max_dist(A.contour, mid)
y = get_max_dist(B.contour, mid)
z = get_max_dist(C.contour, mid)

#### Plotting points

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mp
mp.rcParams['figure.figsize'] = (20,20)

points = np.array([A.centroid,B.centroid,C.centroid])
corners = np.asarray([x,y,z])
corner_marker_contours = np.asarray((conts[0].contour,conts[1].contour,conts[2].contour))

plt.figure()

cv2.drawContours(image, corner_marker_contours, -1, (0, 255, 0), 3)
original = plt.imshow(image)
marker_centroids = plt.scatter(points[:, 0], points[:, 1], s = 50)
marker_corners = plt.scatter(corners[:,0], corners[:,1], s=50)
marker_mid = plt.plot(mid[0], mid[1], 'ro')
plt.show()

![](points.jpg)
<center>Points located for a sample image.</center>
<center><font color="#ff0000">QR centroid(approx)</font> <font color="#ffa500">QR code Coners</font> <font color="#00cd00">Position markers</font> <font color="#3232ff">Position Marker centroid</font> </center>

## Photogammetry

![](opencv.jpg)
<center> Perspective transfomration. [OpenCV.org](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html)</center>

#### Sensor coordinates (u,v)
These are obtained from the given image. We assume image coordinates to have origin at the top left corner of the image sensor. These coordinates can be considered as homogeneous coordinates transformed from image coordinate system.  
 We need to ensure that the ordering of the points is same as the ordering of the points obtained from the image. Here we use the order: top-left, top-right, bottom-left and midpoint.

In [ ]:
image_points = np.array([x, z, y, mid])

#### Real world coordinates (X,Y,0)
The center of the world is considered to be the center of the QR code.  
Since we know that that the marker is lying on the floor and is flat, all the real world Z coordinates for the marker will be 0. This reduces a variable from the Perspective transformation.  
    We know the actual dimensions of the QR code and that it is a square. So the corners will be at distance actual_dimension/2 from the center.

In [ ]:
def get_world_points(actual_dim):
    """
    Returns the world coordinates of the given the actual dimension of the image, 
    assuming that the world center is the object center.
    """
    world_points = np.array([(-actual_dim / 2, actual_dim / 2, 0.0),
                                 (actual_dim / 2, actual_dim / 2, 0.0),
                                 (-actual_dim / 2, -actual_dim / 2, 0.0),
                                 (0,0, 0.0)
                                 ])
    return world_points

#### Intrinsic Matrix
Even though the pinhole camera model does not have a focal length we need it for the scale factor while converting to homgeneous coordinates. The intrinsic camera matrix needs the camera center and the focal length (if we ignore the shear). The camera center can be regarded as the center of the image we have. We can assume the pinhole camera to be a cuboid such that that the distance of the pinhole from the screen (focal length) is equal to either one of the dimensions of the image. Here we consider the height of the image. Another way to determinig the focal length is using camera calibration tool provided in the OpenCV API for even accurate representation.

In [ ]:
def get_intrinsic(image):
    """
    Returns the intrinsic matrix of the camera based on the above assumptions.
    """
    focal_length = image.shape[0] #px=cm
    cam_center_x = image.shape[0] / 2
    cam_center_y = image.shape[1] / 2

    intrinsic = np.array([[focal_length, 0, cam_center_y],
                         [0, focal_length, cam_center_x],
                         [0, 0, 1]
                         ], dtype="double")
    return intrinsic

In [ ]:
def get_cam_vectors(image, image_points, actual_dim, distortion):
    """
    Returns the rotaional and translational vectro of a camera, 
    given the image, known image point and actual dimensions of the object.
    """
    world_points = get_world_points(actual_dim = 8.8) #cm=px
    intrinsic = get_intrinsic(image)
    flag, rotation_vector, translation_vector = cv2.solvePnP(world_points,
                                                         image_points,
                                                         intrinsic,
                                                         distortion,
                                                         flags=cv2.SOLVEPNP_ITERATIVE)
    return rotation_vector, translation_vector

We also assume that there is no distortion. 

In [ ]:
distortion = np.zeros((4, 1))
actual_dim = 8.8
rotation_vector, translation_vector = get_cam_vectors(image, image_points, actual_dim, distortion)

#### Convert the rotational vector to a rotational matrix (R)

In [ ]:
rotation_matrix, jacobian = cv2.Rodrigues(rotation_vector)

#### Finding the pose of the camera in the real world system
If R is the rotational matrix of the camera, C is the camera pose matrix and T is the translational vector, we will equate RC + T to 0 since we need to find the location of the camera with respect to the world origin (0,0,0). We will leverage the fact that transpose of a rotational matrix is its inverse.

=> 0 = RC + T   
=> -RC = T  
=> Rinv(-RC) = RinvT  
=> -C = RtT  
=> C = -RtT  


In [ ]:
pose = np.matmul(-rotation_matrix.transpose(), translation_vector)
pose = pose.squeeze()

#### Camera orientation vector
Assuming that the camera is initially looking up the z-axis, [0, 0, 1]T will be the initial orientation vector of the camera.  
If Or if the final orientation vector of the camera, then Or can be obtained by transforming [0, 0, 1] using the rotational matrix.

=> [0, 0, 1]T = ROr  
=> Rinv[0, 0, 1]T = RinvROr  
=> Rinv[0, 0, 1]T = Or  
=> Rt[0, 0, 1]T = Or  

In [ ]:
orientation = np.matmul(rotation_matrix.T, np.array([0, 0, 1]).T)

In [ ]:
import math
def get_euler_angles(R):
    """
    Converts the rotational matrix to euler angles in degrees.
    (yaw, pitch , roll)
    """
    x = math.atan2(R[2,1] , R[2,2])
    y = math.atan2(-R[2,0], math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0]))
    z = math.atan2(R[1,0], R[0,0])
    return np.degrees(x), np.degrees(y), np.degrees(z)

In [ ]:
angles = get_euler_angles(rotation_matrix)

# Visualization

In [ ]:
def plot_cam(ax, pose, orientation, angles):
    """
    Plotting the camera in real world, given its pose, orientation and angles.
    """
    ax.scatter([pose[0]], [pose[1]], [pose[2]])
    label_pose = ("Pose", '%.3f' % pose[0], '%.3f' % pose[1], '%.3f' % pose[2])
    ax.text(pose[0], pose[1], pose[2], label_pose)
    label_orient = ("Orientation Vector", '%.3f' % orientation[0], '%.3f' % orientation[1], '%.3f' % orientation[2])
    ax.text(pose[0], pose[1], pose[2]+5, label_orient)
    label_angle = ("Yaw,Pitch,Roll", '%.3f' % angles[0], '%.3f' % angles[1], '%.3f' % angles[2])
    ax.text(pose[0], pose[1], pose[2]+10, label_angle)
    ax.plot([pose[0], 0],[pose[1], 0], zs=[pose[2], 0])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
axis_length = max(100, np.amax(pose))
ax.set_xlim3d(-axis_length, axis_length)
ax.set_ylim3d(-axis_length, axis_length)
ax.set_zlim3d(0, axis_length)

# Plot camera
plot_cam(ax,
         np.around(pose, decimals=4),
         np.around(orientation, decimals=4),
         np.around(angles, decimals=4))

# Plot QR Code
pattern = cv2.imread('pattern.jpg')
pattern = cv2.flip(cv2.transpose(pattern), 0)

qr_x, qr_y = np.meshgrid(np.linspace(-actual_dim, actual_dim, pattern.shape[0]),
                     np.linspace(-actual_dim, actual_dim, pattern.shape[0]))
X = qr_x
Y = qr_y
Z = 0
ax.plot_surface(X, Y, Z, facecolors=pattern / 255., shade=False)

plt.gca().set_aspect('equal', adjustable='box')

plt.show()

![](result.jpg)
<center>Final result</center>